<a href="https://colab.research.google.com/github/sohambharadwaj/major_project/blob/main/car_price_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np


In [21]:
car=pd.read_csv('https://raw.githubusercontent.com/sohambharadwaj/major_project/main/quikr_car%20-%20quikr_car.csv')

In [22]:
car.head(5)

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [23]:
car.shape

(892, 6)

In [24]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [25]:
## creating backup of car dataset
backup=car.copy()

Cleaning Data

year has many non-year values

In [26]:
car=car[car['year'].str.isnumeric()]

year is in object. Change to integer

In [27]:
car['year']=car['year'].astype(int)

Price has Ask for Price

In [28]:
car=car[car['Price']!='Ask For Price']

Price has commas in its prices and is in object¶


In [29]:
car['Price']=car['Price'].str.replace(',','').astype(int)

kms_driven has object values with kms at last.

In [30]:
car['kms_driven']=car['kms_driven'].str.split().str.get(0).str.replace(',','')

It has nan values and two rows have 'Petrol' in them

In [31]:
car=car[car['kms_driven'].str.isnumeric()]

In [32]:
car['kms_driven']=car['kms_driven'].astype(int)

fuel_type has nan values

In [33]:
car=car[~car['fuel_type'].isna()]

In [34]:
car.shape

(816, 6)

name and company had spammed data...but with the previous cleaning, those rows got removed

Company does not need any cleaning now. Changing car names. Keeping only the first three words



In [35]:
car['name']=car['name'].str.split().str.slice(start=0,stop=3).str.join(' ')

Resetting the index of the final cleaned data

In [36]:
car=car.reset_index(drop=True)

Cleaned Data

In [37]:
car

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
811,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
812,Tata Indica V2,Tata,2009,110000,30000,Diesel
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
814,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [38]:
car.to_csv('Cleaned_Car_data.csv')

In [39]:
car['kms_driven'].value_counts()

45000     30
35000     29
55000     25
50000     23
20000     21
          ..
29685      1
59910      1
10750      1
30201      1
111111     1
Name: kms_driven, Length: 246, dtype: int64

In [40]:
X=car[['name','company','year','kms_driven','fuel_type']]
y=car['Price']

In [41]:
X

,name,company,year,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,36000,Diesel
4,Ford Figo,Ford,2012,41000,Diesel
...,...,...,...,...,...
811,Maruti Suzuki Ritz,Maruti,2011,50000,Petrol
812,Tata Indica V2,Tata,2009,30000,Diesel
813,Toyota Corolla Altis,Toyota,2009,132000,Petrol
814,Tata Zest XM,Tata,2018,27000,Diesel


In [42]:
y.shape

(816,)

Applying Train Test Split

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [44]:
from sklearn.linear_model import LinearRegression

In [45]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

Creating an OneHotEncoder object to contain all the possible categories

In [46]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

Creating a column transformer to transform categorical columns


In [47]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),
                                    remainder='passthrough')

Linear Regression Model

In [48]:
lr=LinearRegression()

Making a pipeline

In [49]:
pipe=make_pipeline(column_trans,lr)

Fitting the model

In [50]:
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'B...
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)],
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                          

In [51]:

y_pred=pipe.predict(X_test)

Checking R2 Score

In [52]:
r2_score(y_test,y_pred)

0.7153307238885149

Finding the model with a random state of TrainTestSplit where the model was found to give almost 0.92 as r2_score

In [53]:
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [54]:
np.argmax(scores)

144

In [55]:
scores[np.argmax(scores)]

0.921001521164875

In [56]:
pipe.predict(pd.DataFrame(columns=X_test.columns,data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([419628.08114079])

The best model is found at a certain random state

In [57]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

0.921001521164875

In [58]:
import pickle

In [59]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [60]:
pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([444907.62223276])